# Imports

In [1]:
from __future__ import print_function
import time
import os
from datetime import datetime

import numpy as np

# MPI support
from mpi4py import MPI
comm = MPI.COMM_WORLD
mpi_size = comm.Get_size() # number of processes
mpi_rank = comm.Get_rank() # rank of current process
WITH_MPI = mpi_size > 1

# PyNN library
import pyNN.neuron as sim
from pyNN import space
from pyNN.parameters import Sequence
from pyNN.utility import init_logging # connection_plot is bugged
import neo.io

In [ ]:
# Custom PyNN extensions
from bgcellmodels.extensions.pynn.synapses import (
    GluSynapse, GabaSynapse, GabaSynTmHill, GABAAsynTM,
    GabaSynTm2, NativeMultiSynapse)
from bgcellmodels.extensions.pynn.utility import connection_plot
from bgcellmodels.extensions.pynn.populations import Population

# Monkey-patching of pyNN.neuron.Population class
# from bgcellmodels.extensions.pynn.recording import TraceSpecRecorder
# sim.Population._recorder_class = TraceSpecRecorder

# Custom NEURON mechanisms
from bgcellmodels.mechanisms import synapses, noise # loads MOD files

# Custom cell models
import bgcellmodels.models.STN.GilliesWillshaw.gillies_pynn_model as gillies
import bgcellmodels.models.GPe.Gunay2008.gunay_pynn_model as gunay
# import bgcellmodels.models.striatum.Mahon2000_MSN.mahon_pynn_model as mahon
# import bgcellmodels.models.interneuron.Golomb2007_FSI.golomb_pynn_model as golomb

import bgcellmodels.cellpopdata.connectivity as connectivity # for use in config files
ConnectivityPattern = connectivity.ConnectivityPattern
make_connection_list = connectivity.make_connection_list
make_divergent_pattern = connectivity.make_divergent_pattern

# Our physiological parameters
# from bgcellmodels.cellpopdata.physiotypes import Populations as PopID
#from bgcellmodels.cellpopdata.physiotypes import ParameterSource as ParamSrc
# from bgcellmodels.cellpopdata.cellpopdata import CellConnector

from bgcellmodels.common.spikelib import make_oscillatory_bursts, make_variable_bursts
from bgcellmodels.common.configutil import eval_params
from bgcellmodels.common.stdutil import getdictvals
from bgcellmodels.common import logutils, fileutils

# Debug messages
logutils.setLogLevel('quiet', [
    'bpop_ext',
    'bluepyopt.ephys.parameters', 
    'bluepyopt.ephys.mechanisms', 
    'bluepyopt.ephys.morphologies'])

	1 
	1 
 
loading STh cell data structures...
 
	1 
	1 
  2 trees
  loaded tree 0 (23 branches)
  loaded tree 1 (11 branches)
	1 

[LFPsim] Functions in <lfp_lib.hoc> loaded into Hoc


# Simulation Arguments

In [ ]:
# SETPARAM: config file and output dir
config_file = '/home/luye/workspace/bgcellmodels/bgcellmodels/models/network/LuNetSGS/configs/DA-depleted-v3_CTX-f0_template.json'
out_basedir = '~/storage'

In [ ]:
# Parse config JSON file to dict
config_file = os.path.expanduser(config_file)
config_name, ext = os.path.splitext(os.path.basename(config_file))
sim_config = fileutils.parse_json_file(config_file, nonstrict=True)
config = sim_config

In [ ]:
# Post process output specifier
if out_basedir is None or out_basedir == '': # shell can pass empty string
    out_basedir = '~/storage'
job_id = 1
time_now = time.time()
timestamp = datetime.fromtimestamp(time_now).strftime('%Y.%m.%d')

# Default output directory
# NOTE: don't use timestamp -> mpi ranks will make different filenames
out_subdir = '{stamp}_job-{job_id}_{config_name}'.format(
                                        stamp=timestamp,
                                        job_id=job_id,
                                        config_name=config_name)

# File names for data files
# Default output format is hdf5 / NIX io
filespec = '*_{stamp}_test_job-{job_id}.mat'.format(
                                        stamp=timestamp,
                                        job_id=job_id)

# Make output directory if non-existing, but only on one host
out_basedir = os.path.expanduser(out_basedir)
if not os.path.isdir(out_basedir) and mpi_rank == 0:
    os.mkdir(out_basedir)

# Don't make directory with variable timestamp -> mpi ranks will make different
out_fulldir = os.path.join(out_basedir, out_subdir)
if not os.path.isdir(out_fulldir) and mpi_rank == 0:
    os.mkdir(out_fulldir)
output = os.path.join(out_fulldir, filespec)

In [ ]:
# Function arguments
pop_scale       = 1.0
sim_dur         = 500.0
export_locals   = False
with_gui        = True
report_progress = False
seed            = 888
calculate_lfp   = False
burst_frequency = 20.0
transient_period = 0
max_write_interval = 1000.0

# Simulator Setup

In [ ]:
sim.setup(timestep=0.025, min_delay=0.1, max_delay=10.0, use_cvode=False)
if mpi_rank == 0:
    init_logging(logfile=None, debug=True)


print("""\nRunning net on MPI rank {} with following settings:
- sim_dur = {}
- output = {}""".format(mpi_rank, sim_dur, output))

print("\nThis is node {} ({} of {})\n".format(
      sim.rank(), sim.rank() + 1, sim.num_processes()))

h = sim.h
sim.state.duration = sim_dur # not used by PyNN, only by our custom funcs
sim.state.rec_dt = 0.05
sim.state.mcellran4_rng_indices = {} # Keep track of MCellRan4 indices for independent random streams.
finit_handlers = []

# Make one random generator that is shared and should yield same results
# for each MPI rank, and one with unique results.
# - The shared (parallel-safe) RNGs should be used in functions that are
#   executed on all ranks, like instantiating Population and Projection
#   objects.
# - The default RNG for Connectors is NumpyRNG(seed=151985012)
if seed is None:
    seed = config['simulation']['shared_rng_seed']
shared_seed = seed # original: 151985012
sim.state.rank_rng_seed = rank_seed = sim.state.native_rng_baseseed + sim.state.mpi_rank
# RNGs that can be passed to PyNN objects like Connector subclasses
shared_rng_pynn = sim.NumpyRNG(seed=shared_seed)
rank_rng_pynn = sim.NumpyRNG(seed=rank_seed)
# Raw Numpy RNGs (numpy.random.RandomState) to be used in our own functions
shared_rng = shared_rng_pynn.rng
rank_rng = rank_rng_pynn.rng

# Local Functions

In [ ]:
params_global_context = globals()
params_local_context = locals() # capture function arguments

def get_pop_parameters(pop, *param_names):
    """
    Get population parameters from config and evaluate them.
    """
    local_context = config[pop].get('local_context', {})
    param_specs = getdictvals(config[pop], *param_names, as_dict=True)
    pvals = eval_params(param_specs, params_global_context, local_context)
    return getdictvals(pvals, *param_names)

def get_cell_parameters(pop):
    """
    Get PyNN cell parameters as dictionary of numerical values.
    """
    local_context = config[pop].get('local_context', {})
    param_specs = config[pop].get('PyNN_cell_parameters', {})
    return eval_params(param_specs, params_global_context, local_context)

def synapse_from_config(pre, post):
    """
    Make Synapse object from config dict
    """
    local_context = config[post].get('local_context', {})
    syn_type, syn_params = getdictvals(config[post][pre]['synapse'],
                                       'name', 'parameters')
    syn_class = synapse_types[syn_type]
    syn_pvals = eval_params(syn_params, params_global_context, local_context)
    return syn_class(**syn_pvals)

def connector_from_config(pre, post, rng=None):
    """
    Make Connector object from config dict
    """
    config_locals = config[post].get('local_context', {})
    con_type, con_params = getdictvals(config[post][pre]['connector'],
                                       'name', 'parameters')
    connector_class = getattr(sim, con_type)
    con_pvals = eval_params(con_params, params_global_context,
                           [params_local_context, config_locals])
    connector = connector_class(**con_pvals)
    if rng is not None:
        connector.rng = rng
    return connector


# LFP calculation: command line args get priority over config file
if calculate_lfp is None:
    calculate_lfp, = get_pop_parameters('STN', 'calculate_lfp')

# Set NEURON integrator/solver options
if calculate_lfp:
    sim.state.cvode.use_fast_imem(True)
sim.state.cvode.cache_efficient(True) # necessary for lfp, also 33% reduction in simulation time

# Populations

## STN Population

## GPE Population

In [ ]:
# Get common parameters for GPE cells
gpe_dx, gaba_mech, gpe_pop_size, frac_proto, frac_arky = get_pop_parameters(
    'GPE.all', 'grid_dx', 'GABA_mechanism', 'base_population_size',
    'prototypic_fraction', 'arkypallidal_fraction')

gpe_common_params = get_cell_parameters('GPE.all')

gpe_grid = space.Line(x0=0.0, dx=gpe_dx,
                      y=1e6, z=0.0)

In [ ]:
#---------------------------------------------------------------------------
# GPE Prototypic

proto_type = gunay.GpeProtoCellType(**gpe_common_params)
proto_type.model.default_GABA_mechanism = gaba_mech # workaround for string parameter

ncell_proto = int(gpe_pop_size * pop_scale * frac_proto)
pop_gpe_proto = Population(ncell_proto, 
                           cellclass=proto_type,
                           label='GPE.proto',
                           structure=gpe_grid)

In [ ]:
#---------------------------------------------------------------------------
# GPE Arkypallidal

arky_type = gunay.GpeArkyCellType(**gpe_common_params)
arky_type.model.default_GABA_mechanism = gaba_mech # workaround for string parameter

ncell_arky = int(gpe_pop_size * pop_scale * frac_arky)
pop_gpe_arky = Population(ncell_arky, 
                          cellclass=arky_type,
                          label='GPE.arky',
                          structure=gpe_grid)

In [ ]:
#---------------------------------------------------------------------------
# GPE Surrogate spike sources

num_gpe_surrogates, surr_rate = get_pop_parameters('GPE.all', 
    'num_surrogates', 'surrogate_rate')

if num_gpe_surrogates > 0:
    pop_gpe_surrogate = Population(num_gpe_surrogates, 
                                   sim.SpikeSourcePoisson(rate=surr_rate),
                                   label='GPE.surrogate')
else:
    pop_gpe_surrogate = None

In [ ]:
#---------------------------------------------------------------------------
# GPE Assembly (Proto + Arky)

asm_gpe = sim.Assembly(pop_gpe_proto, pop_gpe_arky)

In [ ]:
pop_ctx = Population(10, sim.SpikeSourceArray(spike_times=range(10)),
                         label='CTX')

In [ ]:
[pop for pop in Population.all_populations if isinstance(
                        pop.celltype, sim.cells.NativeCellType)]

## STR Population

## CTX Population